# read light names from MQTT/zigbee2mqtt

In [1]:
import os
import sys
import json
import argparse
from urllib.parse import urlparse

import paho.mqtt.publish as publish
import paho.mqtt.subscribe as subscribe


In [2]:
def get_url_from_configfile(config_path):
    parser = argparse.ArgumentParser()
    for n in ["--host", "--username", "--pw"]:
        parser.add_argument(n, type=str)
    parser.add_argument("--port", type=int)
    parser.add_argument("--custom", nargs=argparse.REMAINDER)

    if not os.path.exists(config_path):
        print("Error: missing config ({})".format(config_path))
        sys.exit(1)

    args = parser.parse_args(
        open(config_path, "r").read().strip().split(), namespace=argparse.Namespace()
    )
    url = urlparse(
        "mqtts://{username}:{pw}@{host}:{port}".format(
            username=args.username, pw=args.pw, host=args.host, port=str(args.port)
        )
    )
    return url


In [3]:
def get_lights(lights_group_name, url):
    topics = [
        "zigbee2mqtt/bridge/devices",
        "zigbee2mqtt/bridge/groups",
    ]

    results = subscribe.simple(
        topics,
        msg_count=len(topics),
        qos=1,
        retained=True,
        hostname=url.hostname,
        port=url.port,
        auth={"username": url.username, "password": url.password},
        tls={},
    )

    devices, groups = sorted(
        {
            msg.topic.removeprefix("zigbee2mqtt/bridge/"): json.loads(msg.payload)
            for msg in results
        }.items()
    )

    target_groups = [d for d in groups[1] if d["friendly_name"] == lights_group_name]
    light_group = target_groups[0]
    light_members = sorted([member["ieee_address"] for member in light_group["members"]])
    light_names = sorted(
        [d["friendly_name"] for d in devices[1] if d["ieee_address"] in light_members]
    )
    return light_names


In [4]:
url = get_url_from_configfile(os.path.expanduser("~/.config/mosquitto_pub"))
light_names = get_lights("lights_all", url)


In [5]:
light_names


['balcony_rgb',
 'bathroom_rgb',
 'bathroom_switch',
 'bedroom_rgb',
 'bedroom_tem',
 'frontroom_switch',
 'kitchen_dim',
 'kitchen_rgb',
 'livingroom_balloon_dim',
 'livingroom_rgb',
 'livingroom_switch',
 'livingroom_tem',
 'paper_lamp_tem_bottom',
 'paper_lamp_tem_middle',
 'paper_lamp_tem_top']